In [20]:
from warnings import filterwarnings

filterwarnings("ignore")

import pandas as pd
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from ta.trend import ema_indicator

from modules.backtester import long_only_backtester
from modules.data_fetcher import download_historical_data
from modules.indicators import fischer_transformation


In [28]:
symbol = "BTC-USDT"

df_BTC = download_historical_data(symbol, "1day")  # .loc["2021-11-20":]
df_BTC.dropna(inplace=True)
# df_BTC = df_BTC.asfreq("H").ffill()
print(df_BTC.shape)
df_BTC.head()


(1892, 7)


,Timestamp,Open,Close,High,Low,Amount,Volume
Date,,,,,,,
2018-01-01 01:00:00,1.514765e+09,12933.000001,13586.890000,14150.000000,12500.000000,29.886527,395120.343363
2018-01-02 01:00:00,1.514851e+09,13586.890000,14100.000000,15650.000000,13000.000000,60.922287,848603.986127
2018-01-03 01:00:00,1.514938e+09,14050.000000,15380.011104,15980.000000,13480.000002,57.435863,847353.744804
2018-01-04 01:00:00,1.515024e+09,15131.011108,15194.999997,15450.000003,14000.100000,59.166551,876400.179632
2018-01-05 01:00:00,1.515110e+09,15194.999997,17266.663939,17350.000000,14501.000000,62.192731,979397.149187


In [29]:
df_BTC["Fischer10"] = fischer_transformation(df_BTC.Close, 10)
df_BTC["EMA160"] = ema_indicator(df_BTC.Close, 160)
df_BTC.dropna(inplace=True)
df_BTC.head()

,Timestamp,Open,Close,High,Low,Amount,Volume,Fischer10,EMA160
Date,,,,,,,,,
2018-06-09 02:00:00,1.528502e+09,7617.534659,7493.999999,7721.899725,7450.000000,79.393057,6.048811e+05,0.807592,9372.378213
2018-06-10 02:00:00,1.528589e+09,7493.999999,6762.999999,7493.999999,6615.000656,434.355849,3.043770e+06,0.131223,9339.963577
2018-06-11 02:00:00,1.528675e+09,6762.999999,6850.329393,7001.289462,6615.000001,300.338636,2.034001e+06,-0.418211,9309.036444
2018-06-12 02:00:00,1.528762e+09,6859.528450,6523.007840,6896.400444,6447.464224,204.372001,1.364128e+06,-0.955288,9274.427393
2018-06-13 02:00:00,1.528848e+09,6524.451296,6299.999998,6612.000000,6100.000000,310.663263,1.986175e+06,-1.462695,9237.477984


In [30]:
high_threashold = 0.5
low_threashold = -0.5


In [31]:
fig = make_subplots(
    rows=2,
    cols=1,
    subplot_titles=("Historical price", "Fischer transformation"),
    shared_xaxes=True,
)

fig.add_trace(
    go.Candlestick(
        name="Historical price",
        x=df_BTC.index,
        open=df_BTC["Open"],
        high=df_BTC["High"],
        low=df_BTC["Low"],
        close=df_BTC["Close"],
    ),
    row=1,
    col=1,
)

fig.add_trace(
    go.Scatter(
        name="Fischer transformation",
        x=df_BTC.index,
        y=df_BTC["Fischer10"],
    ),
    row=2,
    col=1,
)
fig.add_shape(
    type="line",
    x0=df_BTC.index[0].strftime("%Y-%m-%d %H:%M:%S"),
    y0=high_threashold,
    x1=df_BTC.index[-1].strftime("%Y-%m-%d %H:%M:%S"),
    y1=high_threashold,
    line={"color": "Black"},
    xref="x",
    yref="y",
    row=2,
    col=1,
)
fig.add_shape(
    type="line",
    x0=df_BTC.index[0].strftime("%Y-%m-%d %H:%M:%S"),
    y0=low_threashold,
    x1=df_BTC.index[-1].strftime("%Y-%m-%d %H:%M:%S"),
    y1=low_threashold,
    line={"color": "Black"},
    xref="x",
    yref="y",
    row=2,
    col=1,
)
fig.update_layout(
    xaxis_rangeslider_visible=False,
    showlegend=True,
    title_text="Fischer transformation exploration",
)
fig.show()

In [32]:
def buy_func(row: pd.Series, prev_row: pd.Series) -> bool:
    return True if row.Fischer10 <= low_threashold and prev_row.Fischer10 >= low_threashold and row.Close >= row.EMA160 else False


def sell_func(row: pd.Series, prev_row: pd.Series, timeframe_count: int) -> bool:
    return True if row.Fischer10 >= high_threashold and prev_row.Fischer10 <= high_threashold else False


long_only_backtester(df_BTC, buy_func, sell_func, stop_loss=0.05)

-------------  General informations  -------------
Period: [2018-06-09 02:00:00] -> [2023-03-07 01:00:00]
Intial balance: 1000 $

-------------  Strategy performance  -------------
Final balance: 1011.02 $
Final net balance: 957.74 $
Strategy net return: 95.77 %
Buy and Hold return: 298.76 %
Strategy winrate: 40.74 %
Strategy fees: 59.13 $
Strategy volatility: 0.07 %
Sharpe ratio: 0.04 (no risk free rate)
Sharpe ratio: 0.02 (risk free rate = buy and hold)

-------------  Trades informations  --------------
Mean trade duration: 4 days 11:31:06
Total trades: 27
Total good trades: 11
Mean good trades return: 7.90 %
Median good trades return: 8.42 %
Best trades return: 12.62 % | Date: 2021-10-03 02:00:00 | Duration: 9 days 00:00:00
Mean good trade duration: 7 days 04:16:21

Total bad trades: 16
Mean bad trades return: -5.00 %
Median bad trades return: -5.00 %
Worst trades return: -5.00 % | Date: 2019-08-16 02:00:00 | Duration: 1 days 00:00:00
Mean bad trade duration: 2 days 15:00:00
Exit r